In [1]:
import deepxde as dde
import numpy as np

import yaml
import pandas as pd
import time
import datetime
import os

#from deepxde.backend import tf
#from deepxde.backend import torch
import torch
from scipy.special import gamma
from scipy.special import lambertw

#import mayavi.mlab as mlab
import matplotlib.pyplot as plt

#from draw_counter import random_sphere_points, uniform_sample_points
from PIL import Image

torch.set_default_tensor_type(torch.cuda.FloatTensor)

CopyFlag = 0
## 读取Config文件
with open('./Config/train_20220715.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print(config)
config['NetWidth']

## 更新参数
## 单位统一至cm，定义各个参数
# D = 0.585
# Rsd = 17.5573
# #Rwall = 2.654
# Rwall = 2.56401
# rpump = 0.75
# q = 5
# rcell = 1.5

# Rrel = Rsd + q*Rwall
# I = config['LightIntensity']*1e-3*1e4 # kg*(cm)^2/s^3
T = config['Temperature']
# k1 = 10**(-20)*10**8 # kg*cm^4/s^2
# k2 = 10**(-17)*10**4 # cm^2
# k3 = 1 # cm^-3
# sigm = 2.34601*10**(-13) # cm^2

# n = k3*(1/(T+273.15))*10**(21.866+4.402-4453/(T+273.15))

D = 6.9519E-1 #cm²/s
I = 25*10 # kg*(cm)^2/s^3
q = 5
Rwall = 6.4424 #1/s
Rsd = 13.457 #1/s
Rrel = Rsd+q*Rwall

k1 = 10**(-20)*10**8 # kg*cm^4/s^2
k2 = 10**(-17)*10**4 # cm^2
k3 = 1 # cm^-3
sigm = 2.34601*10**(-13) # cm^2
n = 3.7405E13 #1/m³
rcell = 1.5
rpump = 0.75

Using backend: pytorch



{'ConfigDate': datetime.date(2022, 7, 15), 'CSVPath': './result/TrainResult0725', 'LightIntensity': 25, 'Temperature': 170, 'CopyFlag': [1, 2, 3], 'NetDepth': [2, 4, 6, 8], 'NetWidth': [140, 160], 'TrainEpoch': 90000}


In [6]:
'''
读取Ref文件，COMSOL仿真结果
'''
# 读取3D仿真结果文件
filename_3D = './COMSOL_data/COMSOL_3D.csv'
#df = pd.read_csv(filename, header=9, names=['x', 'y', 'z', 'Pz'])
df_3D = pd.read_csv(filename_3D, header=9, names=['x', 'y', 'z', 'Pz'])
axis_3D_x = np.asarray(df_3D['x'])
axis_3D_y = np.asarray(df_3D['y'])
axis_3D_z = np.asarray(df_3D['z'])
axis_3D_Pz = np.asarray(df_3D['Pz'])
axis_3D_axis = np.stack((axis_3D_x, axis_3D_y, axis_3D_z), axis=1)
# # 读取z-axis实验结果
# filename = './COMSOL_data/COMSOL_zaxis.csv'
# #df = pd.read_csv(filename, header=9, names=['x', 'y', 'z', 'Pz'])
# df = pd.read_csv(filename, header=9, names=['Pz'])
# ZAxis_COMSOL = np.asarray(df['Pz'])



# ## PINNs，计算z-axis的极化率数据. z_fitness 拟合结果

# def Z_Axis(sample_num=300, rcell=1.5):
#     x = 0
#     y = 0
#     z_sequence = np.linspace(-rcell, rcell, sample_num)
#     sample_data = []
    
#     for z in z_sequence:
#         sample_data.append((x, y, z))
        
#     sample_data = np.asarray(sample_data, dtype=np.float32)
#     return sample_data


# def fitness_metric(y_pred, y_ref):
#     # 优化度拟合矩阵
#     y_l2_error = np.linalg.norm(y_pred - y_ref, ord=2) # L2 范数
#     y_pred_sum = np.sum(y_pred**2)
    
#     fitness = 1 - np.sqrt(y_l2_error / y_pred_sum)
#     return fitness

def bloch_pde(x_in, y_in):
    ''' x_in: (x,y,z) axis
        y_in: Pz polarization
    '''
    x_in_copy = x_in.detach().cpu().numpy()
    x = x_in_copy[:, 0:1]
    y = x_in_copy[:, 1:2]
    z = x_in_copy[:, 2:3]
    
#     sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph())
#     x_arr = x.eval(session=sess)
    dy_xx = dde.grad.hessian(y_in, x_in, i=0, j=0)
    dy_yy = dde.grad.hessian(y_in, x_in, i=1, j=1)
    dy_zz = dde.grad.hessian(y_in, x_in, i=2, j=2) # dy/dxidxj
    
    plog1 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))-n*sigm*z))
    plog2 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))+n*sigm*z))
    Rop0_z = k2*(Rrel/sigm)*(np.real(plog1) + np.real(plog2))
    Rop0_z = np.asarray(Rop0_z)    
    
    Rop = Rop0_z * np.exp(-2*(x**2+y**2)/rpump**2)
    Rop = torch.from_numpy(Rop).cuda()
    #Rop = Rop0
    
    return -D*(dy_xx+dy_yy+dy_zz) + ((Rop+Rsd)/q+Rwall)*y_in - Rop/q


def func_boundary(x_in, Pz, numpy_X):
    x_in_copy = x_in.detach().cpu().numpy()
    x = x_in_copy[:, 0:1]
    y = x_in_copy[:, 1:2]
    z = x_in_copy[:, 2:3]
    
    dPz_x = dde.grad.jacobian(Pz, x_in, i=0, j=0) # This is right
    dPz_y = dde.grad.jacobian(Pz, x_in, i=0, j=1)
    dPz_z = dde.grad.jacobian(Pz, x_in, i=0, j=2)
    
    #Rop = Rop0 * np.exp(-2*(x**2+y**2)/rpump**2)
    
    plog1 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))-n*sigm*z))
    plog2 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))+n*sigm*z))
    Rop0_z = k2*(Rrel/sigm)*(np.real(plog1) + np.real(plog2))
    Rop0_z = np.asarray(Rop0_z)      
    
    Rop = Rop0_z * np.exp(-2*(x**2+y**2)/rpump**2)
    
    second_term = np.sqrt(1/2*np.abs(Rop*D))
    
    # tranfer back to torch.tensor
    second_term = torch.from_numpy(second_term).cuda()
    Rop = torch.from_numpy(Rop).cuda()
    x = torch.from_numpy(x).cuda()
    y = torch.from_numpy(y).cuda()
    z = torch.from_numpy(z).cuda()
    
    return (D/rcell*(x*dPz_x + y*dPz_y + z*dPz_z) + Pz*second_term)

In [7]:
## 自定义callbacks, 每1000epoch，打印变量的值

from deepxde.callbacks import Callback
import os
import torch

class printVariable(Callback):
    def __init__(self, var_list, period=1000, filename=None, precision=2):
        super().__init__()
        self.var_list = var_list if isinstance(var_list, list) else [var_list]
        self.period = period
        self.precision = precision

        #self.file = sys.stdout if filename is None else open(filename, "w", buffering=1)
        self.value = None
        self.epochs_since_last = 0
        self.model = None
    
    def on_train_begin(self):

        self.value = [var.detach().item() for var in self.var_list]
        print(
            'Variable pred',
            self.value
        )
    
    def on_epoch_end(self):
        self.epochs_since_last += 1
        if self.epochs_since_last >= self.period:
            self.epochs_since_last = 0
            self.on_train_begin()

    def get_value(self):
        """Return the variable values."""
        return self.value
    
    def set_model(self, model):
        if model is not self.model:
            self.model = model
            self.init()
   
class saveModel(Callback):
    def __init__(self, saveDir, saveName, period=1000, filename=None, precision=2):
        super().__init__()
        #self.var_list = var_list if isinstance(var_list, list) else [var_list]
        self.period = period
        self.precision = precision

        #self.file = sys.stdout if filename is None else open(filename, "w", buffering=1)
        self.value = None
        self.epochs_since_last = 0
        self.model = None
        self.modelNum = 0
        self.saveDir = saveDir
        self.saveName = saveName
        
    def set_model(self, model):
        if model is not self.model:
            self.model = model
            self.init()    
            
    def on_epoch_end(self):
        self.epochs_since_last += 1
        if self.epochs_since_last >= self.period:
            self.modelNum += 1
            self.epochs_since_last = 0  
            if not os.path.exists(self.saveDir):
                #os.mkdir(self.saveDir)
                os.makedirs(self.saveDir)
            savePath = os.path.join(self.saveDir, self.saveName+str(self.modelNum)+'.pt')
            #print(self.model.net.state_dict())
            checkpoint = {
                "model_state_dict": self.model.net.state_dict(),
                "optimizer":self.model.opt.state_dict()
            }
            torch.save(checkpoint, savePath)            

class loadModel(Callback):
    def __init__(self, loadPath):
        super().__init__()
        #self.var_list = var_list if isinstance(var_list, list) else [var_list]
        self.loadPath = loadPath
        
    def set_model(self, model):
        if model is not self.model:
            self.model = model
            self.init()    
            
    def on_train_begin(self):
        checkpoint = torch.load(self.loadPath)
        self.model.net.load_state_dict(checkpoint["model_state_dict"])
        self.model.opt.load_state_dict(checkpoint["optimizer"])
        print('Load model successful')
        
class EarlyStoping(Callback):
    def __init__(self,sigma=5e-4):
        super().__init__()
        #self.var_list = var_list if isinstance(var_list, list) else [var_list]
        self.epochs_since_last = 0
        self.sigma = sigma
        
    def on_epoch_end(self):
        self.epochs_since_last += 1
        train_loss = sum(self.model.train_state.loss_train)
        if train_loss < self.sigma:
            self.model.stop_training = True
            print('Early stoping')
    

In [8]:
geom = dde.geometry.Sphere([0, 0, 0], rcell)
bc = dde.icbc.OperatorBC(geom, func_boundary, lambda _, on_boundary: on_boundary) # 函数调研下，operatorBC

data = dde.data.PDE(geom, bloch_pde, bc, num_domain=20000, num_boundary=2000)
ModelSaveDir = './model/forward/'
if not os.path.exists(ModelSaveDir):
    os.makedirs(ModelSaveDir)
## 核心训练程序
for depth in config['NetDepth']:
    for width in config['NetWidth']:
        net = dde.nn.FNN([3] + [width] * depth + [1], "tanh", "Glorot normal")
        # net.apply_output_transform(
        #     lambda x, y: (1 - tf.reduce_sum(x ** 2, axis=1, keepdims=True)) * y
        # )
        myCallback1 = saveModel(ModelSaveDir,'pre',period=10000)
        myCallback2 = EarlyStoping()
        model = dde.Model(data, net)
        model.compile("adam", lr=1e-4)
        #losshistory, train_state = model.train(epochs=8000, model_save_path='Bloch_Rop_v1')
        losshistory, train_state = model.train(epochs=config['TrainEpoch'], callbacks=[myCallback1, myCallback2])
        print('Train finish : {0} width {1} depth'.format(width, depth))

        YZ_dir_path = './result/Forward/{4}/{5}/Intensity{0}_Temp{1}_NetDepth{2}_Width{3}'.format(config['LightIntensity'], T, depth,\
                                                                                    width,config['CSVPath'],CopyFlag)
        if not os.path.exists(YZ_dir_path):
            os.makedirs(YZ_dir_path)
#         zAxis_pred = model.predict(Z_Axis())[:, 0]
#         zAxis_df = pd.DataFrame(zAxis_pred)
#         zAxis_path = os.path.join(YZ_dir_path, 'zAxis_pred.csv')
#         zAxis_df.to_csv(zAxis_path)       
#         z_fitness = fitness_metric(zAxis_pred, ZAxis_COMSOL)

        ## 计算三维数据预测结果
        y_pred_3D = model.predict(axis_3D_axis)[:, 0]
        y_ref = axis_3D_Pz

        #L2_mean = np.linalg.norm(y_pred_3D - y_ref, ord=2) # 开了跟号
        L2_error = np.mean(np.sqrt((y_pred_3D-y_ref)**2))
        print('Evaluation : ', L2_error)
        ## 按照实验时间，保存实验结果
        date_time =  datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') # 2022-06-18 13:49
        record_path = config['CSVPath'] + str(config['CopyFlag'][CopyFlag]) + '.csv'

        # if not os.path.exists(record_path):
        #     os.mkdir('./result/')

        EvaluateDf = pd.DataFrame({
            'Date': [date_time],
            'Epoch': [config['TrainEpoch']],
            'Intensity(mW)': [config['LightIntensity']],
            'Temperature' : [config['Temperature']],
            'Network depth' : [depth],
            'Network width' : [width],
            #'Z axis fitness' : [z_fitness],
            'L2 mean error': [L2_error]
        })    

        EvaluateDf.to_csv(record_path, index=False, mode='a')

        ## 保存截面数据, X=0的YZ平面，坐标轴生成

        sample_num = 200

        def plane_YZ(x=0, sample_num = 200):
            radius_x = np.sqrt(rcell**2 -x**2)
            sample_data = []    

            for theta in np.linspace(0, 2*np.pi, sample_num):
                for r_pow in np.linspace(0, radius_x**2, sample_num):
                    y = np.sqrt(r_pow) * np.cos(theta)
                    z = np.sqrt(r_pow) * np.sin(theta)
                    sample_data.append((x, y, z)) # 200*200 = 40000个采样点

            sample_data = np.asarray(sample_data, dtype=np.float32
                                    )
            return sample_data

        YZ_axis = plane_YZ(0)
        y_pred_YZ = model.predict(YZ_axis)[:, 0]
        y_pred_YZ = np.reshape(y_pred_YZ, (sample_num, sample_num))
        YZ_df = pd.DataFrame(y_pred_YZ.T)
        YZ_file_path = os.path.join(YZ_dir_path, 'Plane_YZ_Epoch_{}.csv'.format(config['TrainEpoch']))    
        YZ_df.to_csv(YZ_file_path)   

/usr/local/lib/python3.6/dist-packages/skopt/sampler/sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+20002=20002. 
  total_n_samples))


Compiling model...
'compile' took 0.000653 s

Training model...

Step      Train loss              Test loss               Test metric
0         [7.38e+01, 2.93e-01]    [7.38e+01, 2.93e-01]    []  


/usr/local/lib/python3.6/dist-packages/skopt/sampler/sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+2002=2002. 
  total_n_samples))


1000      [6.12e+00, 1.24e+00]    [6.12e+00, 1.24e+00]    []  
2000      [4.02e-01, 1.74e+00]    [4.02e-01, 1.74e+00]    []  
3000      [2.59e-01, 1.67e+00]    [2.59e-01, 1.67e+00]    []  
4000      [2.30e-01, 1.60e+00]    [2.30e-01, 1.60e+00]    []  
5000      [2.30e-01, 1.47e+00]    [2.30e-01, 1.47e+00]    []  
6000      [2.42e-01, 1.34e+00]    [2.42e-01, 1.34e+00]    []  
7000      [2.73e-01, 1.17e+00]    [2.73e-01, 1.17e+00]    []  
8000      [3.10e-01, 8.86e-01]    [3.10e-01, 8.86e-01]    []  
9000      [2.96e-01, 5.19e-01]    [2.96e-01, 5.19e-01]    []  
10000     [2.15e-01, 2.55e-01]    [2.15e-01, 2.55e-01]    []  
11000     [1.59e-01, 1.07e-01]    [1.59e-01, 1.07e-01]    []  
12000     [1.24e-01, 5.06e-02]    [1.24e-01, 5.06e-02]    []  
13000     [1.07e-01, 3.42e-02]    [1.07e-01, 3.42e-02]    []  
14000     [9.67e-02, 2.57e-02]    [9.67e-02, 2.57e-02]    []  
15000     [8.28e-02, 1.95e-02]    [8.28e-02, 1.95e-02]    []  
16000     [7.32e-02, 1.58e-02]    [7.32e-02, 1.58e-02] 

34000     [1.14e-02, 5.51e-03]    [1.14e-02, 5.51e-03]    []  
35000     [1.09e-02, 5.35e-03]    [1.09e-02, 5.35e-03]    []  
36000     [1.05e-02, 5.19e-03]    [1.05e-02, 5.19e-03]    []  
37000     [1.02e-02, 5.04e-03]    [1.02e-02, 5.04e-03]    []  
38000     [1.01e-02, 4.93e-03]    [1.01e-02, 4.93e-03]    []  
39000     [9.58e-03, 4.77e-03]    [9.58e-03, 4.77e-03]    []  
40000     [9.19e-03, 4.63e-03]    [9.19e-03, 4.63e-03]    []  
41000     [9.09e-03, 4.52e-03]    [9.09e-03, 4.52e-03]    []  
42000     [8.98e-03, 4.36e-03]    [8.98e-03, 4.36e-03]    []  
43000     [8.41e-03, 4.26e-03]    [8.41e-03, 4.26e-03]    []  
44000     [8.45e-03, 4.16e-03]    [8.45e-03, 4.16e-03]    []  
45000     [1.44e-02, 4.25e-03]    [1.44e-02, 4.25e-03]    []  
46000     [7.89e-03, 3.92e-03]    [7.89e-03, 3.92e-03]    []  
47000     [7.53e-03, 3.80e-03]    [7.53e-03, 3.80e-03]    []  
48000     [7.34e-03, 3.69e-03]    [7.34e-03, 3.69e-03]    []  
49000     [1.07e-02, 3.69e-03]    [1.07e-02, 3.69e-03] 

Compiling model...
'compile' took 0.000308 s

Training model...

Step      Train loss              Test loss               Test metric
0         [7.01e+01, 6.14e-02]    [7.01e+01, 6.14e-02]    []  
1000      [2.22e-01, 1.63e+00]    [2.22e-01, 1.63e+00]    []  
2000      [2.13e-01, 1.43e+00]    [2.13e-01, 1.43e+00]    []  
3000      [1.55e-01, 6.15e-01]    [1.55e-01, 6.15e-01]    []  
4000      [1.83e-01, 4.93e-01]    [1.83e-01, 4.93e-01]    []  
5000      [1.38e-01, 4.68e-01]    [1.38e-01, 4.68e-01]    []  
6000      [4.83e-01, 2.41e-01]    [4.83e-01, 2.41e-01]    []  
7000      [1.05e-01, 1.79e-01]    [1.05e-01, 1.79e-01]    []  
8000      [7.84e-02, 5.12e-02]    [7.84e-02, 5.12e-02]    []  
9000      [2.95e-02, 1.60e-02]    [2.95e-02, 1.60e-02]    []  
10000     [1.92e-02, 7.05e-03]    [1.92e-02, 7.05e-03]    []  
11000     [1.45e-02, 4.12e-03]    [1.45e-02, 4.12e-03]    []  
12000     [1.05e-02, 1.95e-03]    [1.05e-02, 1.95e-03]    []  
13000     [5.74e-03, 1.39e-03]    [5.74e-03, 1

In [9]:
checkpoint = {
    "model_state_dict": model.net.state_dict(),
    "optimizer":model.opt.state_dict()
}
torch.save(checkpoint, 'pre1.pt')    

In [7]:
def Sample_PlaneData(z=0, sample_num = 200):
    radius_z = np.sqrt(rcell**2 -z**2)
    sample_data = []    

    for theta in np.linspace(0, 2*np.pi, sample_num):
        for r_pow in np.linspace(0, radius_z**2, sample_num):
            x = np.sqrt(r_pow) * np.cos(theta)
            y = np.sqrt(r_pow) * np.sin(theta)
            sample_data.append((x, y, z)) # 200*200 = 40000个采样点

    sample_data = np.asarray(sample_data, dtype=np.float32
                            )
    return sample_data

SampleAxis = Sample_PlaneData()
SampleDf = pd.DataFrame(SampleAxis).to_csv('axis_position.csv',index=None,header=False)
SampleAxis

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 1.0633218e-01,  0.0000000e+00,  0.0000000e+00],
       [ 1.5037641e-01,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 1.4924433e+00, -3.6554318e-16,  0.0000000e+00],
       [ 1.4962264e+00, -3.6646978e-16,  0.0000000e+00],
       [ 1.5000000e+00, -3.6739403e-16,  0.0000000e+00]], dtype=float32)